In [218]:
# %%shell
# git clone --branch TrggTin --single-branch https://github.com/vphuhan/21KHDL-TikTok-Analytics.git
# cd 21KHDL-TikTok-Analytics
# git sparse-checkout init --cone
# git sparse-checkout set data/interim
# git checkout

In [219]:
# pip install pandas nltk underthesea scikit-learn tqdm

# Imports and Initialization

In [220]:
import pandas as pd
import re
import unicodedata
import nltk
from underthesea import word_tokenize, pos_tag, ner
from sklearn.feature_extraction.text import TfidfVectorizer
from difflib import get_close_matches
import logging
import json
import os
from tqdm import tqdm
import string
import regex as re
import traceback
import jdc  
from spellchecker import SpellChecker
from datetime import datetime

In [221]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("extraction_log.log"),
        logging.StreamHandler()
    ]
)

In [222]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# VietnameseTextProcessor Class Definition

In [223]:
class VietnameseTextProcessor:
    def __init__(self, food_list_path=None, location_list_path=None):
        """
        Khởi tạo Bộ xử lý văn bản tiếng Việt

        Tham số:
            food_list_path (str): Đường dẫn đến tệp JSON chứa danh sách món ăn Việt Nam
            location_list_path (str): Đường dẫn đến tệp JSON chứa danh sách địa điểm ở Việt Nam
        """
        # Tải hoặc khởi tạo danh sách món ăn và địa điểm
        self.foods = self._load_entity_list(food_list_path, "foods")
        self.locations = self._load_entity_list(location_list_path, "locations")

        # Các từ khóa phổ biến liên quan đến món ăn và hương vị trong tiếng Việt để hỗ trợ nhận diện
        self.food_indicators = [
            "bánh", "phở", "bún", "xèo", "cơm", "gỏi", "chả", "xôi", "cao lầu", "cháo",
            "mì", "hủ tiếu", "nem", "ram", "khọt",
            "lẩu", "cá", "thịt", "canh", "rau", "đậu", "nướng", "ốc", "súp", "bắp",
            "chuối", "nộm", "trà", "cà phê", "sinh tố", "kem", "tàu hủ", "chè", "yaourt", "nước mía",
            "sữa", "kẹo", "đa", "nem chua", "gà", "món", "ăn"
        ]

        self.taste_indicators = [
            "ngon", "ngọt", "chua", "cay", "đắng", "mặn", "bùi", "béo", "giòn", "mềm",
            "thơm", "nồng", "đậm đà", "nhạt", "thanh", "tươi", "ướp", "rim", "kho", "xào",
            "nướng", "luộc", "hấp", "chiên", "xốt", "tẩm", "ướt", "khô", "giòn tan", "dai",
            "sần sật", "mọng nước", "đắng nghét", "chát", "cay xè", "tê", "mặn chát", "ngọt lịm", "béo ngậy", "thơm lừng",
            "nồng nàn", "đậm vị", "nhạt nhẽo", "thanh mát", "tươi rói", "tươi ngon", "đậm đà hương vị", "vừa ăn", "hợp khẩu vị"
        ]

        self.locations_indicators = [
            "Quận", "Huyện", "Phường", "Xã", "Thành phố", "TP", "Tỉnh", "đường", "phố", "chợ", "địa chỉ", "nằm ở", "tại",
            "Quận 1", "Quận 2", "Quận 3", "Quận 4", "Quận 5", "Quận 6", "Quận 7", "Quận 8", "Quận 9", "Quận 10",
            "Quận 11", "Quận 12", "Bình Thạnh", "Tân Bình", "Tân Phú", "Phú Nhuận", "Gò Vấp", "Bình Tân", "Thủ Đức", "Hóc Môn",
            "Củ Chi", "Nhà Bè", "Cần Giờ", "Bình Chánh", "TP Thủ Đức",
            "Hà Nội", "Hồ Chí Minh", "Đà Nẵng", "Hải Phòng", "Cần Thơ", "Huế", "Nha Trang", "Vũng Tàu", "Đà Lạt",
            "Hạ Long", "Mỹ Tho", "Long Xuyên", "Rạch Giá", "Cà Mau", "Biên Hòa", "Buôn Ma Thuột", "Thái Nguyên", "Nam Định"
        ]

        # Tải các tài nguyên của NLTK nếu cần
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

        # Tạo thư mục để lưu trữ các tệp dữ liệu được trích xuất
        os.makedirs("extracted_data", exist_ok=True)


# Helper Methods

In [224]:
def _load_entity_list(self, file_path, entity_type):
    """Tải danh sách thực thể từ tệp hoặc trả về tập rỗng mặc định"""
    if file_path and os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return set(json.load(f))
        except Exception as e:
            logging.warning(f"Lỗi khi tải danh sách {entity_type}: {e}")

    logging.info(f"Không tìm thấy danh sách {entity_type} hiện có, bắt đầu với tập rỗng")
    return set()

def save_entity_list(self, entity_list, entity_type):
    """Lưu danh sách thực thể đã cập nhật vào tệp"""
    file_path = f"extracted_data/{entity_type}_list.json"
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(list(entity_list), f, ensure_ascii=False, indent=2)
    logging.info(f"Đã lưu {len(entity_list)} {entity_type} vào {file_path}")

def normalize_vietnamese_text(self, text):
    """Chuẩn hóa văn bản tiếng Việt bằng cách xử lý dấu và chữ hoa/thường"""
    if not isinstance(text, str):
        return ""

    # Chuẩn hóa ký tự Unicode
    text = unicodedata.normalize('NFC', text)

    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def clean_text(self, text):
    """Làm sạch văn bản bằng cách loại bỏ ký tự đặc biệt và chuẩn hóa"""
    if not isinstance(text, str):
        return ""

    # Chuẩn hóa văn bản
    text = self.normalize_vietnamese_text(text)

    # Loại bỏ đường dẫn URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Loại bỏ biểu tượng cảm xúc và ký tự đặc biệt trong khi giữ lại chữ tiếng Việt
    # Biểu thức chính quy này giữ lại chữ cái tiếng Việt, số, dấu câu và khoảng trắng
    vietnamese_pattern = r'[^\p{L}\p{N}\p{P}\s]+'
    text = re.sub(vietnamese_pattern, '', text, flags=re.UNICODE)

    # Sửa khoảng cách xung quanh dấu câu
    text = re.sub(r'\s+([.,;:?!])', r'\1', text)

    return text

def auto_correct_text(self, text):
    """Tự động sửa lỗi chính tả bằng bộ kiểm tra chính tả"""
    spell = SpellChecker(language='vi')
    words = word_tokenize(text)
    corrected_words = [spell.correction(word) for word in words]
    return " ".join(corrected_words)

def load_stopwords(self, file_path):
    """Tải danh sách từ dừng từ tệp"""
    if file_path and os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return set(f.read().splitlines())
        except Exception as e:
            logging.warning(f"Lỗi khi tải danh sách từ dừng: {e}")
    logging.info("Không tìm thấy tệp từ dừng, bắt đầu với tập rỗng")
    return set()

def remove_stopwords(self, text, stopwords):
    """Loại bỏ từ dừng khỏi văn bản"""
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return " ".join(filtered_words)

def preprocess_text(self, text):
    """Áp dụng tất cả các bước tiền xử lý lên văn bản"""
    try:
        text = self.clean_text(text)
        text = self.auto_correct_text(text)  # Đã sửa lỗi tại đây
        stopwords = self.load_stopwords('vietnamese-stopwords.txt')
        text = self.remove_stopwords(text, stopwords)
        return text
    except Exception as e:
        logging.error(f"Lỗi khi tiền xử lý văn bản: {e}")
        return text if isinstance(text, str) else ""

# Gán các phương thức vào lớp VietnameseTextProcessor
VietnameseTextProcessor._load_entity_list = _load_entity_list
VietnameseTextProcessor.save_entity_list = save_entity_list 
VietnameseTextProcessor.normalize_vietnamese_text = normalize_vietnamese_text
VietnameseTextProcessor.clean_text = clean_text
VietnameseTextProcessor.auto_correct_text = auto_correct_text
VietnameseTextProcessor.load_stopwords = load_stopwords
VietnameseTextProcessor.preprocess_text = preprocess_text


# Entity Extraction Methods

In [225]:
def extract_entities_from_ner(self, text):
    """Trích xuất thực thể từ văn bản bằng Named Entity Recognition (NER) của underthesea."""
    locations = []

    try:
        ner_tags = ner(text)  # Thực hiện nhận dạng thực thể có tên (NER)

        # Kiểm tra nếu kết quả từ NER có định dạng mong đợi
        if not isinstance(ner_tags, list):
            return locations

        # Trích xuất các địa điểm từ NER
        current_loc = []

        for item in ner_tags:
            # Xử lý các định dạng đầu ra khác nhau từ NER
            if isinstance(item, (list, tuple)) and len(item) == 2:
                word, tag = item
            else:
                continue

            if tag.startswith('B-LOC'):
                if current_loc:
                    locations.append(' '.join(current_loc))
                    current_loc = []
                current_loc.append(word)
            elif tag.startswith('I-LOC') and current_loc:
                current_loc.append(word)
            elif current_loc:
                locations.append(' '.join(current_loc))
                current_loc = []

        # Thêm thực thể địa điểm cuối cùng nếu có
        if current_loc:
            locations.append(' '.join(current_loc))

    except Exception as e:
        logging.error(f"Lỗi khi trích xuất thực thể bằng NER: {e}")
        logging.error(traceback.format_exc())

    return locations

def extract_entities_from_patterns(self, text, sentences, pos_tags):
    """Trích xuất thực thể bằng cách sử dụng phương pháp dựa trên mẫu (Pattern Matching)."""
    foods = []
    locations = []
    tastes = []

    # Xử lý từng câu để trích xuất thực thể
    for idx, sentence in enumerate(sentences):
        words = word_tokenize(sentence)
        sentence_pos_tags = pos_tags[idx] if idx < len(pos_tags) else []

        # Tìm thực thể về thực phẩm
        self._extract_food_entities(sentence, sentence_pos_tags, foods)

        # Tìm thực thể về địa điểm
        self._extract_location_entities(sentence, sentence_pos_tags, locations)

        # Tìm mô tả về hương vị
        self._extract_taste_descriptions(sentence, words, tastes)

    return foods, locations, tastes

def _extract_food_entities(self, sentence, pos_tags, foods):
    """Trích xuất thực thể thực phẩm từ một câu."""
    # Kiểm tra danh sách thực phẩm có sẵn
    for food in self.foods:
        if food.lower() in sentence.lower():
            foods.append(food)

    # Tìm các từ chỉ thực phẩm
    for idx, (word, tag) in enumerate(pos_tags):
        if word.lower() in self.food_indicators:
            noun_phrase = [word]
            for i in range(1, 4):
                if idx + i < len(pos_tags):
                    next_word, next_tag = pos_tags[idx + i]
                    if next_tag.startswith(('N', 'A')):  # Danh từ hoặc Tính từ
                        noun_phrase.append(next_word)
                    else:
                        break
            
            if noun_phrase:
                food_name = " ".join(noun_phrase)
                foods.append(food_name)
                self.foods.add(food_name)

def _extract_location_entities(self, sentence, pos_tags, locations):
    """Trích xuất thực thể địa điểm từ một câu."""
    # Kiểm tra danh sách địa điểm có sẵn
    for location in self.locations:
        if location.lower() in sentence.lower():
            locations.append(location)

    # Tìm các từ chỉ địa điểm
    for idx, (word, tag) in enumerate(pos_tags):
        if any(indicator.lower() in word.lower() for indicator in self.locations_indicators):
            noun_phrase = [word]
            for i in range(1, 4):
                if idx + i < len(pos_tags):
                    next_word, next_tag = pos_tags[idx + i]
                    if next_tag.startswith(('N', 'M', 'Np')):  # Danh từ, Số, Danh từ riêng
                        noun_phrase.append(next_word)
                    else:
                        break
            
            if noun_phrase:
                location_name = " ".join(noun_phrase)
                locations.append(location_name)
                self.locations.add(location_name)

def _extract_taste_descriptions(self, sentence, words, tastes):
    """Trích xuất mô tả về hương vị từ một câu."""
    for taste_word in self.taste_indicators:
        if taste_word in sentence.lower():
            taste_idx = -1
            for idx, word in enumerate(words):
                if taste_word in word.lower():
                    taste_idx = idx
                    break
            
            if taste_idx >= 0:
                start = max(0, taste_idx - 3)
                end = min(len(words), taste_idx + 4)
                taste_phrase = " ".join(words[start:end])
                tastes.append(taste_phrase)

def extract_entities(self, text):
    """Trích xuất các thực thể về thực phẩm, địa điểm và hương vị từ văn bản."""
    if not text or not isinstance(text, str):
        return {"foods": [], "locations": [], "tastes": []}

    try:
        results = {"foods": [], "locations": [], "tastes": []}

        # Trích xuất địa điểm bằng NER
        ner_locations = self.extract_entities_from_ner(text)
        results["locations"].extend(ner_locations)
        self.locations.update(ner_locations)

        # Trích xuất thực thể bằng phương pháp dựa trên mẫu
        sentences = nltk.sent_tokenize(text)
        pos_tags = [pos_tag(sent) for sent in sentences]

        foods, locations, tastes = self.extract_entities_from_patterns(text, sentences, pos_tags)

        results["foods"].extend(foods)
        results["locations"].extend(locations)
        results["tastes"].extend(tastes)

        # Cập nhật danh sách thực thể
        self.foods.update(foods)
        self.locations.update(locations)

        # Loại bỏ trùng lặp và lọc bỏ chuỗi rỗng
        for key in results:
            results[key] = list(set(filter(None, results[key])))

        return results

    except Exception as e:
        logging.error(f"Lỗi khi trích xuất thực thể: {e}")
        logging.error(traceback.format_exc())
        return {"foods": [], "locations": [], "tastes": []}
    
VietnameseTextProcessor.extract_entities_from_ner = extract_entities_from_ner
VietnameseTextProcessor.extract_entities_from_patterns = extract_entities_from_patterns
VietnameseTextProcessor._extract_food_entities = _extract_food_entities
VietnameseTextProcessor._extract_location_entities = _extract_location_entities
VietnameseTextProcessor._extract_taste_descriptions = _extract_taste_descriptions
VietnameseTextProcessor.extract_entities = extract_entities


# DataFrame Processing and Bootstrapping

In [226]:
def process_dataframe(self, df, text_column="video_transcription", batch_size=100):
    """
    Xử lý toàn bộ DataFrame và trích xuất các thực thể.

    Tham số:
        df (pd.DataFrame): DataFrame chứa dữ liệu văn bản.
        text_column (str): Tên cột chứa văn bản.
        batch_size (int): Kích thước batch để xử lý nhằm tiết kiệm bộ nhớ.

    Trả về:
        pd.DataFrame: DataFrame gốc với các cột chứa thực thể được trích xuất.
    """
    # Kiểm tra nếu DataFrame trống hoặc không có cột văn bản
    if df.empty or text_column not in df.columns:
        logging.error(f"DataFrame không hợp lệ hoặc thiếu cột '{text_column}'")
        return df

    # Tạo thư mục lưu trữ nếu chưa tồn tại
    os.makedirs("extracted_data", exist_ok=True)

    # Khởi tạo các cột để lưu thực thể trích xuất
    df['preprocessed_text'] = ""
    df['extracted_foods'] = None
    df['extracted_locations'] = None
    df['extracted_tastes'] = None

    total_batches = (len(df) + batch_size - 1) // batch_size  # Tính số batch cần xử lý

    for i in tqdm(range(total_batches), desc="Đang xử lý batch"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))

        batch = df.iloc[start_idx:end_idx].copy()

        # Tiền xử lý văn bản
        batch['preprocessed_text'] = batch[text_column].apply(self.preprocess_text)

        # Trích xuất thực thể
        entities_list = []
        for text in batch['preprocessed_text']:
            entities_list.append(self.extract_entities(text))

        # Cập nhật DataFrame với thực thể trích xuất
        batch['extracted_foods'] = [data['foods'] for data in entities_list]
        batch['extracted_locations'] = [data['locations'] for data in entities_list]
        batch['extracted_tastes'] = [data['tastes'] for data in entities_list]

        # Cập nhật vào DataFrame gốc
        df.iloc[start_idx:end_idx] = batch

        # Lưu kết quả tạm thời theo từng batch
        if (i + 1) % 5 == 0 or (i + 1) == total_batches:
            self.save_entity_list(self.foods, "foods")
            self.save_entity_list(self.locations, "locations")

            # Lưu kết quả trung gian
            checkpoint_file = f"extracted_data/processed_data_batch_{i+1}.csv"
            df.iloc[:end_idx].to_csv(checkpoint_file, index=False)
            logging.info(f"Đã lưu kết quả trung gian vào {checkpoint_file} sau batch {i+1}/{total_batches}")

    # Thống kê số lượng thực thể đã tìm thấy
    food_count = len(self.foods)
    location_count = len(self.locations)

    logging.info(f"Trích xuất hoàn tất. Tìm thấy {food_count} thực thể món ăn và {location_count} thực thể địa điểm.")

    return df

def bootstrap_entity_lists(self, df, text_column="preprocessed_text", min_freq=3):
    """
    Mở rộng danh sách thực thể bằng TF-IDF để tìm các thực thể tiềm năng.
    
    Tham số:
        df (pd.DataFrame): DataFrame chứa dữ liệu văn bản.
        text_column (str): Tên cột chứa văn bản đã tiền xử lý.
        min_freq (int): Số lần xuất hiện tối thiểu để xem xét một thực thể.

    Trả về:
        set: Tập hợp các thực thể món ăn mới được nhận diện.
    """
    if df.empty or text_column not in df.columns:
        logging.error(f"Không thể mở rộng thực thể: DataFrame không hợp lệ hoặc thiếu cột '{text_column}'")
        return set()

    # Lọc ra các văn bản hợp lệ
    valid_texts = df[text_column].dropna().replace('', pd.NA).dropna().tolist()

    if not valid_texts:
        logging.warning("Không tìm thấy văn bản hợp lệ để mở rộng thực thể")
        return set()

    try:
        min_df_val = max(1, min(min_freq, len(valid_texts) // 2))
        
        tfidf = TfidfVectorizer(
            ngram_range=(1, 3),  # Xét các n-gram từ 1 đến 3 từ
            min_df=min_df_val,  # Điều chỉnh min_df
            max_df=0.9  # Loại bỏ các cụm từ quá phổ biến
        )

        tfidf_matrix = tfidf.fit_transform(valid_texts)
        feature_names = tfidf.get_feature_names_out()

        # Lấy danh sách n-gram có giá trị TF-IDF cao
        important_ngrams = []
        for i in range(min(tfidf_matrix.shape[0], 100)):
            feature_index = tfidf_matrix[i,:].nonzero()[1]
            tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
            # Sắp xếp theo điểm TF-IDF giảm dần
            for idx, score in sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:20]:
                important_ngrams.append(feature_names[idx])

        # Lọc các cụm từ có thể là tên món ăn (dựa vào từ chỉ món ăn)
        potential_foods = set()
        for text in valid_texts:
            for indicator in self.food_indicators:
                if indicator in text:
                    for ngram in important_ngrams:
                        # Kiểm tra nếu ngram xuất hiện gần từ chỉ món ăn
                        if ngram in text and re.search(r'\b' + re.escape(indicator) + r'.{0,30}' + re.escape(ngram), text, re.IGNORECASE):
                            potential_foods.add(ngram)
                        if ngram in text and re.search(r'\b' + re.escape(ngram) + r'.{0,30}' + re.escape(indicator), text, re.IGNORECASE):
                            potential_foods.add(ngram)

        # Lọc bỏ các thực thể không hợp lệ (quá ngắn, chỉ chứa số, v.v.)
        filtered_foods = {food for food in potential_foods if len(food) > 2 and not food.isdigit()}

        # Cập nhật danh sách món ăn
        self.foods.update(filtered_foods)
        logging.info(f"Đã thêm {len(filtered_foods)} thực thể món ăn tiềm năng từ mở rộng thực thể")

        return filtered_foods

    except Exception as e:
        logging.error(f"Lỗi khi mở rộng thực thể: {e}")
        logging.error(traceback.format_exc())
        return set()

VietnameseTextProcessor.process_dataframe = process_dataframe
VietnameseTextProcessor.bootstrap_entity_lists = bootstrap_entity_lists

In [227]:
def main():
    try:
        # Tạo một thể hiện của bộ xử lý văn bản
        processor = VietnameseTextProcessor()

        # Tải tập dữ liệu
        logging.info("Đang tải tập dữ liệu...")
        try:
            # df = pd.read_csv("/content/21KHDL-TikTok-Analytics/data/interim/small_video_transcription.csv")
            df = pd.read_csv("C:/Users/nguye/OneDrive/Tài liệu/GitHub/21KHDL-TikTok-Analytics/data/interim/small_video_transcription.csv")
            if df.empty:
                logging.error("Tập dữ liệu được tải về trống")
                return
            logging.info(f"Tập dữ liệu đã tải có {len(df)} dòng")
        except Exception as e:
            logging.error(f"Lỗi khi tải tập dữ liệu: {e}")
            logging.error(traceback.format_exc())
            return

        # Xử lý một mẫu nhỏ để kiểm thử (sử dụng .head(10) để thử nghiệm, xóa bỏ để xử lý toàn bộ)
        sample_df = df.head(2)

        # Xử lý dữ liệu văn bản
        logging.info("Bắt đầu xử lý văn bản và trích xuất thực thể...")
        processed_df = processor.process_dataframe(sample_df, text_column='video_transcription')

        # Mở rộng danh sách thực thể bằng phương pháp bootstrapping
        logging.info("Thực hiện bootstrapping để mở rộng danh sách thực thể...")
        processor.bootstrap_entity_lists(processed_df)

        # Lưu kết quả cuối cùng
        processed_df.to_csv("extracted_data/fully_processed_data.csv", index=False)
        processor.save_entity_list(processor.foods, "foods")
        processor.save_entity_list(processor.locations, "locations")

        # Lưu kết quả có cấu trúc dưới dạng JSON gồm video_id, author_id và các thực thể trích xuất
        structured_data = []
        for _, row in processed_df.iterrows():
            structured_data.append({
                'video_id': row.get('video_id', ''),
                'author_id': row.get('author_id', ''),
                'extracted_entities': {
                    'foods': row.get('extracted_foods', []),
                    'locations': row.get('extracted_locations', []),
                    'tastes': row.get('extracted_tastes', [])
                }
            })

        with open("extracted_data/structured_entities.json", 'w', encoding='utf-8') as f:
            json.dump(structured_data, f, ensure_ascii=False, indent=2)

        logging.info("Quá trình xử lý hoàn tất. Kết quả đã được lưu trong thư mục 'extracted_data'.")

    except Exception as e:
        logging.error(f"Lỗi nghiêm trọng trong hàm main: {e}")
        logging.error(traceback.format_exc())

In [228]:
if __name__ == "__main__":
    main()

2025-03-09 00:22:05,955 - INFO - Không tìm thấy danh sách foods hiện có, bắt đầu với tập rỗng
2025-03-09 00:22:05,956 - INFO - Không tìm thấy danh sách locations hiện có, bắt đầu với tập rỗng
2025-03-09 00:22:05,957 - INFO - Đang tải tập dữ liệu...
2025-03-09 00:22:06,203 - INFO - Tập dữ liệu đã tải có 10673 dòng
2025-03-09 00:22:06,203 - INFO - Bắt đầu xử lý văn bản và trích xuất thực thể...
C:\Users\nguye\AppData\Local\Temp\ipykernel_16656\74775411.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed_text'] = ""
C:\Users\nguye\AppData\Local\Temp\ipykernel_16656\74775411.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 